In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium .webdriver.chrome.options import Options
import time

In [3]:
opts = Options()
opts.add_argument(" --headless")
chrome_driver = os.getcwd() + r'\driver\chromedriver.exe'
driver = webdriver.Chrome(options=opts, executable_path= chrome_driver)


In [4]:
def download_mutation(pango):
    driver.get('https://outbreak.info/situation-reports?pango=' + pango)
    soup_file  = driver.page_source
    soup = BeautifulSoup(soup_file)
    print(pango + " Downloaded From " + soup.title.get_text())
    
    table = soup.find('table' )
    print
    return pd.read_html(str(table))[0]
    

In [5]:
'https://outbreak.info/situation-reports?pango=' + varient

NameError: name 'varient' is not defined

In [18]:
varient_list = ['P.1','P.1.5','P.1.1','P.1.4']
covid_table = pd.DataFrame({'gene':[np.nan], 'amino acid':[np.nan], 'varient':[np.nan]})
count = 0
final = 0
failed = []         
while varient_list != []:
    varient = varient_list[-1]
    driver.implicitly_wait(100)
    try:
        driver.get('https://outbreak.info/situation-reports?pango=' + varient)
        if count == 0:
            print(varient," [", end= "")
    except Exception:
        print("Warning: Check the internet connection")
        break
    soup_file  = driver.page_source
    soup = BeautifulSoup(soup_file)
 
    table = soup.find(id = 'mutation-table' )
    if table:
        count = 0
        print("] 100% Completed")
        mutation_table = pd.read_html(str(table))[0]
        mutation_table['varient'] = varient
        covid_table = pd.concat([covid_table,mutation_table],axis=0)
        varient_list.remove(varient)
        mutation_table.to_csv(r'out/covid/'+ varient + '.csv')
    else:
        count += 1
        if count % 2 == 0:
            time.sleep(count * 1.5)
            print("|",end='')
        if count == 10:
            failed.append(varient)
            varient_list.remove(varient)
            print("] XXXX Failed! ")
            count = 0
            continue
        print("===",end='')
    if failed != []:
        varient_list.extend(failed)
        final += 1
        if final == 4:
            break
        
        
failed

P.1.4  [===|======|======|===] 100% Completed
P.1.1  [===|======|======|===] 100% Completed
P.1.5  [===|======|======|======] 100% Completed
P.1  [===|======|======] 100% Completed


[]

In [14]:
covid_table 

,gene,amino acid,varient
0,NaN,NaN,NaN
0,ORF1a,A429V,P.1.3
1,ORF1a,S1188L,P.1.3
2,ORF1a,T1754I,P.1.3
3,ORF1a,T1794I,P.1.3
...,...,...,...
18,ORF8,S84L,P.1
19,ORF8,E92K,P.1
20,N,P80R,P.1
21,N,R203K,P.1
